In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import building_blocks
from building_blocks import *

from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from datasets import EliaSolarDataset
from utils import ConfigSettings

In [2]:
cf = ConfigSettings(config_path='config.ini')

In [3]:
solar_dataset = EliaSolarDataset(
        csv_path=cf.data.file_path,
        datetime_column='DateTime',
        target_column="Corrected Upscaled Measurement [MW]",
        context_length=cf.model.context_length,
        frequency=cf.data.frequency,
        train_test_split_year=cf.data.train_test_split_year,
        train_val_split_year=cf.data.train_val_split_year,)

In [4]:
print(solar_dataset.data.shape)

torch.Size([105769, 1, 120])


In [5]:
df = solar_dataset.get_dataframe(preprocessed=True)

In [6]:
df.head()

,DateTime,Corrected Upscaled Measurement [MW]
0,2012-02-01 00:00:00,0.0
1,2012-02-01 01:00:00,0.0
2,2012-02-01 02:00:00,0.0
3,2012-02-01 03:00:00,0.0
4,2012-02-01 04:00:00,0.0


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [8]:
solar_dataset.train_test_split_index

78287

In [9]:

indices = list(range(len(solar_dataset)))
train_indices = indices[:solar_dataset.train_val_split_index]
val_indices = indices[solar_dataset.train_val_split_index:solar_dataset.train_test_split_index]
test_indices = indices[solar_dataset.train_test_split_index:]

# Creating data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size, 
                                        sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size,
                                                sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(solar_dataset, batch_size=cf.model.batch_size,sampler=test_sampler)

In [10]:
# https://stackoverflow.com/questions/65996797/how-to-refresh-a-python-import-in-a-jupyter-notebook-cell
from importlib import reload
reload(building_blocks)
from building_blocks import *

model_params = TimeSeriesTransformerParams(
    input_dim=cf.model.context_length,
    embed_size=cf.model.embedding_size,
    num_layers=cf.model.num_layers,
    heads=cf.model.num_attention_heads,
    device=device,
    forward_expansion=cf.model.forward_expansion,
    dropout=cf.model.dropout,
    forecast_size=cf.model.forecast_size
)
scenario_params = ScenarioParams(
    name="electricity",
    device=device,
    epochs=100,
    dataloader_train=train_loader,
    dataloader_validation=validation_loader,
    dataloader_test=test_loader,
    base_path="/dne" if cf.runtime.run_in_colab else "./"
)
model = TimeSeriesTransformer.from_params(model_params).to(device)
scenario = Scenario(scenario_params)
result = scenario.execute(model)
print(f"execution done")

c:\Repositories\OpenUniversiteit\dne\code\venv\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


checking paths, base path is ./
training and validating the model
<class 'torch.utils.data.dataloader.DataLoader'>
<class 'dict'>
batch: {'data': tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.3918e-03,
          0.0000e+00, 0.0000e+00]],

        [[3.8513e-02, 4.7760e-03, 1.7285e-06,  ..., 2.6904e-01,
          1.8530e-01, 1.0474e-01]],

        [[1.1420e-01, 4.6143e-02, 1.2184e-02,  ..., 2.7145e-02,
          2.5299e-02, 2.2934e-02]],

        ...,

        [[9.5154e-02, 2.0496e-01, 3.1055e-01,  ..., 0.0000e+00,
          1.8110e-03, 2.9419e-02]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.8561e-03,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 7.7026e-02,
          3.1738e-02, 4.0703e-03]]], dtype=torch.float16), 'label': tensor([0.0000e+00, 3.8513e-02, 1.1420e-01, 0.0000e+00, 0.0000e+00, 4.4189e-02,
        0.0000e+00, 3.1128e-02, 1.9177e-01, 3.4888e-01, 3.2300e-01, 0.0000e+00,
        2.7442e-04, 4.1168e-02, 2.7954e-01,

AttributeError: 'str' object has no attribute 'to'